# 1. CONVERTING TXT TO DATAFRAME

In [39]:
import pandas as pd
pd.set_option('display.max_rows',50)

In [40]:
list_of_companies = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL']

### Function no.1
def dataframe_creation(list_of_companies):
    dataframes = []
    for ticker in list_of_companies:
        df = pd.read_csv("{}.us.txt".format(ticker))
        df['Date'] = pd.to_datetime(df['Date'])
        df['Ticker'] = ticker
        dataframes.append(df)  
    
    full_df = pd.concat(dataframes)
    full_df.set_index(['Ticker', 'Date'], inplace=True)
    return full_df

def date_cutoff(df, last_x_years):
    # Get the maximum date from the index level 'Date'
    max_date = df.index.get_level_values('Date').max()
    cutoff_date = max_date - pd.Timedelta(days=(last_x_years * 365))
    df_last_years = df[df.index.get_level_values('Date') >= cutoff_date]
    return df_last_years

    


In [41]:
full_df = dataframe_creation(list_of_companies)

In [42]:
limited_df = date_cutoff(full_df,10)

In [44]:
cutoff_df = limited_df.copy()

In [45]:
cutoff_df['Daily Return'] = cutoff_df.groupby(level='Ticker')['Close'].pct_change()*100
cutoff_df['MA_7'] = cutoff_df.groupby(level='Ticker')['Close'].rolling(window=7).mean().reset_index(level=0, drop=True)
cutoff_df['MA_30'] = cutoff_df.groupby(level='Ticker')['Close'].rolling(window=30).mean().reset_index(level=0, drop=True)
cutoff_df['Volatility_30'] = cutoff_df.groupby(level='Ticker')['Daily Return'].rolling(window=30).std().reset_index(level=0, drop=True)

In [48]:
cutoff_df.groupby(level = 'Ticker')['Daily Return'].mean()

Ticker
AAPL     0.102232
AMZN     0.134511
GOOGL    0.062689
MSFT     0.057675
TSLA     0.210083
Name: Daily Return, dtype: float64

In [49]:
## we can see max is for TSLA and its value is 0.21%

In [50]:
cutoff_df.groupby(level = 'Ticker')['Month']['Volatility_30'].max()

Ticker
AAPL     6.695492
AMZN     6.942335
GOOGL    5.552184
MSFT     6.081448
TSLA     9.445034
Name: Volatility_30, dtype: float64

In [54]:
df = cutoff_df.reset_index()


,Ticker,Date,Open,High,Low,Close,Volume,OpenInt,Daily Return,MA_7,MA_30,Volatility_30
0,AAPL,2007-11-13,20.615,21.897,19.691,21.765,484373501,0,NaN,NaN,NaN,NaN
1,AAPL,2007-11-14,22.733,22.739,20.970,21.274,403585172,0,-2.255915,NaN,NaN,NaN
2,AAPL,2007-11-15,21.280,21.717,20.528,21.040,414487458,0,-1.099934,NaN,NaN,NaN
3,AAPL,2007-11-16,21.193,21.388,20.405,21.309,385660112,0,1.278517,NaN,NaN,NaN
4,AAPL,2007-11-19,21.262,21.540,20.758,20.996,321694225,0,-1.468863,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11925,GOOGL,2017-11-06,1049.100,1052.590,1042.000,1042.680,913954,0,-0.696197,1039.725714,999.831000,1.126278
11926,GOOGL,2017-11-07,1049.650,1053.410,1043.000,1052.390,1303832,0,0.931254,1042.400000,1003.663000,1.130847
11927,GOOGL,2017-11-08,1050.050,1062.690,1047.050,1058.290,1214469,0,0.560629,1045.994286,1006.942667,1.066469
11928,GOOGL,2017-11-09,1048.000,1050.880,1035.850,1047.720,1793994,0,-0.998781,1048.091429,1009.706333,1.092982


In [55]:
df['YearMonth'] = df['Date'].dt.to_period('M')
df

,Ticker,Date,Open,High,Low,Close,Volume,OpenInt,Daily Return,MA_7,MA_30,Volatility_30,YearMonth
0,AAPL,2007-11-13,20.615,21.897,19.691,21.765,484373501,0,NaN,NaN,NaN,NaN,2007-11
1,AAPL,2007-11-14,22.733,22.739,20.970,21.274,403585172,0,-2.255915,NaN,NaN,NaN,2007-11
2,AAPL,2007-11-15,21.280,21.717,20.528,21.040,414487458,0,-1.099934,NaN,NaN,NaN,2007-11
3,AAPL,2007-11-16,21.193,21.388,20.405,21.309,385660112,0,1.278517,NaN,NaN,NaN,2007-11
4,AAPL,2007-11-19,21.262,21.540,20.758,20.996,321694225,0,-1.468863,NaN,NaN,NaN,2007-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11925,GOOGL,2017-11-06,1049.100,1052.590,1042.000,1042.680,913954,0,-0.696197,1039.725714,999.831000,1.126278,2017-11
11926,GOOGL,2017-11-07,1049.650,1053.410,1043.000,1052.390,1303832,0,0.931254,1042.400000,1003.663000,1.130847,2017-11
11927,GOOGL,2017-11-08,1050.050,1062.690,1047.050,1058.290,1214469,0,0.560629,1045.994286,1006.942667,1.066469,2017-11
11928,GOOGL,2017-11-09,1048.000,1050.880,1035.850,1047.720,1793994,0,-0.998781,1048.091429,1009.706333,1.092982,2017-11


In [68]:
monthly_volatility = df.groupby(['Ticker', 'YearMonth'])['Daily Return'].std()
monthly_volatility

Ticker  YearMonth
AAPL    2007-11      1.812362
        2007-12      1.885933
        2008-01      3.710928
        2008-02      2.909737
        2008-03      2.756659
                       ...   
TSLA    2017-07      2.922188
        2017-08      2.182641
        2017-09      2.205271
        2017-10      1.986738
        2017-11      2.797449
Name: Daily Return, Length: 574, dtype: float64

In [64]:
monthly_volatility.groupby(['Ticker'])['Daily Return'].max()

Ticker
AAPL      6.066281
AMZN      6.625269
GOOGL     5.389841
MSFT      5.904519
TSLA     28.836213
Name: Daily Return, dtype: float64